In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive2 = GoogleDrive(gauth)

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
#change: file id, name of csv file 

#find the file id of the corpus.csv file in your drive. This is YOUR file, not mine 
#this can be found by creating a shareable link for the file in your drive. The id is the last part of the link that comes after id = ... 
downloaded = drive2.CreateFile({'id':'1L27TAppAHigMpkpV1eCC3GbvLjz6TN7_'}) 
downloaded.GetContentFile('corpus.csv')  
file_dictionaryy = pd.read_csv('corpus.csv')

In [0]:
file_dictionaryy.columns

Index([''reagan_speeches_028.txt'', ' 'reagan_speeches_000.txt'',
       ' 'reagan_speeches_014.txt'', ' 'reagan_speeches_015.txt'',
       ' 'reagan_speeches_001.txt'', ' 'reagan_speeches_029.txt'',
       ' 'reagan_speeches_017.txt'', ' 'reagan_speeches_003.txt'',
       ' 'reagan_speeches_002.txt'', ' 'reagan_speeches_016.txt'',
       ...
       ' 'GWBUSH_121399.txt'', ' 'GWBUSH_011500.txt'', ' 'GWBUSH_101700.txt'',
       ' 'GWBUSH_021500.txt'', ' 'GWBUSH_011000.txt'', ' 'GWBUSH_100300.txt'',
       ' 'GWBUSH_010700.txt'', ' 'GWBUSH_120299.txt'', ' 'GWBUSH_120699.txt'',
       ' 'GWBUSH_012600.txt''],
      dtype='object', length=340)

In [0]:
# file = file_dictionary.values   ## this cell is for putting all the files in a dictionary shape.
# file_dictionary = {}
# for i in file:
#   file_dictionary[i[0]] = i[1]
!pwd
!cd drive;ls

files_final = []

for file in file_dictionaryy.columns:
  files_final.append(file.replace(" ", "").replace("'", ""))

file_dictionary = {}

for file in files_final:
  with open('/content/drive/My Drive/data88/' + file, 'r') as f:
    text=f.read()
    file_dictionary[file] = text

/content
'My Drive'


In [0]:
#change: text file name to one of your txt files in corpus.csv
print(file_dictionary)
#check with one of your text files to see if it works 
file_dictionary['trump_speeches_048.txt'][1:30] ## Now we can check the key to be the file name, the content is matched to the key

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



'he President. Thank you very '

get help from: https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [0]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

In [0]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
#change: add your languages! 

### Get the stopwords for the languages of your corpus from nltk package
english_stopwords = stopwords.words('english')
corpus_stopwords = ["footnote", "footnotes","decrease", "cruz", "chapter", "chapters", "livingstone", 
                    "meantime", "edward", "illustration", "lieut"] #add stopwords unique to your corpus 


In [0]:
## This is a function designed to remove all the stopwords in our text
def preprocess(text): # one little problem, gensim.utils.simple_preprocess gets rid off "I" automatically. 
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
          if token not in english_stopwords:
            if token not in corpus_stopwords:
              #continue nesting stopwords for different languagses here 
                result.append(token)
    return result

In [0]:
for key in file_dictionary:
  #print("working on file "+ key)
  file_dictionary[key] = preprocess(file_dictionary[key])

In [0]:
#change: the name of the text file to one of your files 
test_file = file_dictionary['trump_speeches_048.txt'] # an example to show what the file is stored in the dictionrary after preprocessed.
test_file

['president',
 'thank',
 'wanted',
 'begin',
 'mentioning',
 'nominee',
 'secretary',
 'department',
 'labor',
 'alex',
 'acosta',
 'degree',
 'harvard',
 'school',
 'great',
 'student',
 'clerk',
 'justice',
 'samuel',
 'alito',
 'tremendous',
 'career',
 'member',
 'member',
 'national',
 'labor',
 'relations',
 'board',
 'senate',
 'confirmation',
 'times',
 'confirmed',
 'alex',
 'wished',
 'best',
 'spoke',
 'going',
 'think',
 'tremendous',
 'secretary',
 'labor',
 'probably',
 'heard',
 'little',
 'mick',
 'mulvaney',
 'congressman',
 'approved',
 'weeks',
 'late',
 'weeks',
 'weeks',
 'late',
 'office',
 'management',
 'budget',
 'think',
 'fantastic',
 'addition',
 'paul',
 'singer',
 'left',
 'know',
 'paul',
 'involved',
 'anti',
 'trump',
 'trump',
 'paul',
 'left',
 'given',
 'total',
 'support',
 'unification',
 'unifying',
 'party',
 'hopefully',
 'going',
 'able',
 'unify',
 'country',
 'important',
 'talking',
 'long',
 'time',
 'important',
 'want',
 'thank',
 'paul',

In [0]:
processed_files = []
for key in file_dictionary:
  processed_files.append(file_dictionary[key])

In [0]:
dictionary = gensim.corpora.Dictionary(processed_files)

In [0]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [0]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_files]

In [0]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [0]:
#change: set topic number and word number to whatever you want 

topic_number = 40 # how many topic you want
word_number = 5

In [0]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=topic_number, id2word=dictionary, passes=2, workers=2)

In [0]:
for idx, topic in lda_model.show_topics(num_topics= topic_number, num_words= word_number, log=False, formatted=True): ## change num_words to see number of words you want to see in a topic
    print('Topic: {} \nWords: {}'.format(idx+1, topic))

Topic: 1 
Words: 0.007*"applause" + 0.004*"iraq" + 0.004*"trump" + 0.003*"russia" + 0.003*"senator"
Topic: 2 
Words: 0.014*"trump" + 0.006*"russia" + 0.005*"minister" + 0.005*"prime" + 0.005*"iran"
Topic: 3 
Words: 0.011*"korea" + 0.009*"north" + 0.006*"china" + 0.005*"tonight" + 0.004*"trump"
Topic: 4 
Words: 0.010*"health" + 0.010*"applause" + 0.007*"insurance" + 0.004*"companies" + 0.004*"clinton"
Topic: 5 
Words: 0.007*"europe" + 0.007*"bosnia" + 0.006*"mission" + 0.006*"nato" + 0.005*"troops"
Topic: 6 
Words: 0.004*"israel" + 0.004*"marines" + 0.004*"negotiations" + 0.004*"soviet" + 0.003*"israeli"
Topic: 7 
Words: 0.005*"tonight" + 0.004*"governor" + 0.004*"ford" + 0.003*"applause" + 0.003*"soviet"
Topic: 8 
Words: 0.023*"senator" + 0.019*"dole" + 0.008*"clinton" + 0.007*"reform" + 0.006*"education"
Topic: 9 
Words: 0.013*"trump" + 0.007*"israel" + 0.005*"iran" + 0.005*"syria" + 0.005*"region"
Topic: 10 
Words: 0.010*"applause" + 0.006*"poland" + 0.005*"education" + 0.004*"trump"

In [0]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx+1, topic))

Topic: 1 
Words: 0.007*"applause" + 0.004*"iraq" + 0.004*"trump" + 0.003*"russia" + 0.003*"senator" + 0.003*"billion" + 0.003*"happen" + 0.002*"iran" + 0.002*"north" + 0.002*"campaign"
Topic: 2 
Words: 0.014*"trump" + 0.006*"russia" + 0.005*"minister" + 0.005*"prime" + 0.005*"iran" + 0.005*"china" + 0.004*"billion" + 0.004*"nato" + 0.004*"meeting" + 0.003*"tremendous"
Topic: 3 
Words: 0.011*"korea" + 0.009*"north" + 0.006*"china" + 0.005*"tonight" + 0.004*"trump" + 0.003*"social" + 0.003*"japan" + 0.003*"south" + 0.003*"weapons" + 0.003*"federal"
Topic: 4 
Words: 0.010*"health" + 0.010*"applause" + 0.007*"insurance" + 0.004*"companies" + 0.004*"clinton" + 0.004*"john" + 0.004*"talking" + 0.004*"actually" + 0.003*"campaign" + 0.003*"everybody"
Topic: 5 
Words: 0.007*"europe" + 0.007*"bosnia" + 0.006*"mission" + 0.006*"nato" + 0.005*"troops" + 0.005*"health" + 0.004*"soviet" + 0.004*"weapons" + 0.003*"agreement" + 0.003*"secure"
Topic: 6 
Words: 0.004*"israel" + 0.004*"marines" + 0.004*"

In [0]:
top = {}
for i in range(topic_number): 
  top.update({i+1:[]})

In [0]:
for key in file_dictionary:
  topics = sorted(lda_model[bow_corpus[processed_files.index(file_dictionary[key])]], key = lambda tup: -1*tup[1])
  for i in topics:
    topic_number = i[0]+1
    inside = [key, i[1]]
    if i[1] > 0.25: ## change the threshold for topic weights
      top[topic_number].append(inside)

Then we make a dictionary (top) for the topics (topics) have been generated for each document (key) in the document set (fiile_dictionary)

In [0]:
Source = []
Target = []
Source_weight = []
Target_weight = []
Topic = []

In [0]:
for i in range(1, topic_number+1):
  for j in range(len(top[i])):
    for k in range(j+1, len(top[i])):
      if top[i][j][1] >= top[i][k][1]:
        Source.append(top[i][j][0])
        Target.append(top[i][k][0])
        Source_weight.append(top[i][j][1])
        Target_weight.append(top[i][k][1])
      else:
        Source.append(top[i][k][0])
        Target.append(top[i][j][0])
        Source_weight.append(top[i][k][1])
        Target_weight.append(top[i][j][1])
      Topic.append(i)

In [0]:
result = pd.DataFrame({'Source':Source, 'Target':Target, 'Source_Weight':Source_weight, 'Target_Weight': Target_weight, 'Topic': Topic})

In [0]:
Type = ['directed'] * len(result)

In [0]:
result = pd.DataFrame({'Source':Source, 'Target':Target, 'Source_Weight':Source_weight, 'Target_Weight': Target_weight, 'Type': Type, 'Topic': Topic})

In [0]:
result 

,Source,Target,Source_Weight,Target_Weight,Type,Topic
0,reagan_speeches_000.txt,reagan_speeches_015.txt,0.563758,0.373891,directed,2
1,reagan_speeches_000.txt,reagan_speeches_048.txt,0.563758,0.394815,directed,2
2,reagan_speeches_000.txt,reagan_speeches_053.txt,0.563758,0.303269,directed,2
3,reagan_speeches_032.txt,reagan_speeches_000.txt,0.691571,0.563758,directed,2
4,clinton_speeches_020.txt,reagan_speeches_000.txt,0.607379,0.563758,directed,2
...,...,...,...,...,...,...
6588,GWBUSH_011500.txt,BUSH_101184.txt,0.942778,0.584306,directed,38
6589,GWBUSH_011000.txt,BUSH_101184.txt,0.739567,0.584306,directed,38
6590,GWBUSH_011500.txt,BUSH_092588.txt,0.942778,0.869031,directed,38
6591,BUSH_092588.txt,GWBUSH_011000.txt,0.869031,0.739567,directed,38


How many rows can a Google Sheet have?
Originally Answered: How many rows can you have in a google spreadsheet before performance becomes a serious problem? Short answer: 400,000 cells - http://docs.google.com/support/b... - Note the distinction between rows and cells. This means that you could have 40,000 rows of 10 columns or 80,000 rows of 5 columns.Oct 12, 2010

In [0]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [0]:
#removed typo in file name, needs to be My\ Drive not My Drive (backslash neccessary)

result.to_csv('topic_edge_list.csv', index=False)
!cp topic_edge_list.csv drive/My\ Drive/ #change to your directory